In [1]:
import pickle
import pandas as pd
import matplotlib
import os
import re

import scipy

import collections
import datetime
import time

import geopandas as gpd

import numpy as np
 
from difflib import get_close_matches

from fuzzywuzzy import process
from fuzzywuzzy import fuzz
from sklearn import linear_model
import statsmodels.api as sm

from linearmodels import PanelOLS, FamaMacBeth
from scipy import stats

import itertools


## Grab Data

In [2]:
os.getcwd()

'/Users/brianreed/Documents/supplyChain/extremes/extremesAnalysisCode'

In [3]:
# goodsData = pd.read_csv("../goodsData_suppliers.csv").drop(columns = {'Unnamed: 0'})
# goodsData = pd.read_csv("../goodsData_igData.csv").drop(columns = {'Unnamed: 0'})
goodsData = pd.read_csv("../utilitiesData_igData.csv").drop(columns = {'Unnamed: 0'})

# goodsData = pd.read_csv("../goodsData.csv").drop(columns = {'Unnamed: 0'})
goodsData = goodsData[~goodsData.revenueChange.isna() & 
                     ~goodsData.costChange.isna()]

# goodsData['precip5Days_annualquant_1x5Yrs'] = 1*(goodsData['precip5Days_annualquant_1x5Yrs'] > 0)

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# goodsData

Let's grab a list with all the relevant variable names. Just keep the core names and we'll grab the lags as we go through it.

Look at the full combinations of variables and whatnot, if we'd like.

In [24]:
'''
print(goodsData.columns[0:50])

cutoffVars  = ['0.95','1xQtr','1xYr','1x5Qtrs','1x5Yrs'] # ,'1x10Qtrs','1x10Yrs'
weatherVars = ['precip_','temp_','precip5Days_','temp5Days_']
statVars    = ['annualquant_','zipquant_','zipQuarterquant_']

outcomeVars = ['lnCost','lnRev','lnCostNormd','lnRevNormd']
# ['incomeChange', 'revenueChange', 'costChange'],'lnCost','lnInc','lnRev']
# ['lnCost','lnRev','lnCostNormd','lnRevNormd']


for weatherVar in weatherVars:
    for statVar in statVars:
        for cutoffVar in cutoffVars:
            indVar = weatherVar + statVar + cutoffVar
            
            for outcomeVar in outcomeVars:
                print(indVar,"~", outcomeVar)'''

'\nprint(goodsData.columns[0:50])\n\ncutoffVars  = [\'0.95\',\'1xQtr\',\'1xYr\',\'1x5Qtrs\',\'1x5Yrs\'] # ,\'1x10Qtrs\',\'1x10Yrs\'\nweatherVars = [\'precip_\',\'temp_\',\'precip5Days_\',\'temp5Days_\']\nstatVars    = [\'annualquant_\',\'zipquant_\',\'zipQuarterquant_\']\n\noutcomeVars = [\'lnCost\',\'lnRev\',\'lnCostNormd\',\'lnRevNormd\']\n# [\'incomeChange\', \'revenueChange\', \'costChange\'],\'lnCost\',\'lnInc\',\'lnRev\']\n# [\'lnCost\',\'lnRev\',\'lnCostNormd\',\'lnRevNormd\']\n\n\nfor weatherVar in weatherVars:\n    for statVar in statVars:\n        for cutoffVar in cutoffVars:\n            indVar = weatherVar + statVar + cutoffVar\n            \n            for outcomeVar in outcomeVars:\n                print(indVar,"~", outcomeVar)'

Focus on the 5 day variables now (we'll later need to edit the loops above to account for this as well, but we can do this for now.) Make them 0/1.

# Direct Effects
Look at the effects on the suppliers when they're affected directly.

The below gives us the full, clustered standard errors.

In [26]:
cutoffVarsYr = ['0.95']  #, ] # ,'1xQtr''1x5Qtrs',
weatherVars  = ['precip_', 'temp_'] #, 'temp5Days_', 'precip5Days_'] # , 'precip_']#, , ] #[,]
statVarsYr   = ['zipquant_', 'ffquant_', 'zipQuarterquant_', 'indQuarterquant_'] #  , , ]  #,'zipQuarterquant_']
outcomeVars  = ['lnRevNormd', 'lnCostNormd']# , 'lnRev', 'lnCost', 'revenueChange', 'costChange']

goodsData = goodsData[~goodsData.lnRev.isna() & ~goodsData.lnCost.isna()] # & ~goodsData.lnCostNormd.isna()]


start = time.time()

results = pd.DataFrame()

i = 0
for outcomeVar in outcomeVars:
    for weatherVar in weatherVars:
        for statVar in statVarsYr:                     
            for cutoffVar in cutoffVarsYr:
                i = i + 1
                indVar = weatherVar + statVar + cutoffVar
                
                
                print(outcomeVar, "~", indVar)


                # find: concurrent ; or lagged supplier data
                X = goodsData.loc[:,((goodsData.columns.str.contains(indVar)) | 
                                                (goodsData.columns.str.contains('indQtr_')) |
                                                (goodsData.columns.str.contains('gvkey_')))]#  | 
                                                # (goodsData.columns.str.contains('firmQtr_')) |
                                                #(goodsData.columns.str.contains('ageQtr_')) |
                                                #(goodsData.columns.str.contains('sizeQtr_')) |
                                                #(goodsData.columns.str.contains('profitQtr_')))]
                
                
                firms = goodsData['gvkey']


                y = goodsData[outcomeVar]
                
                
                model = sm.OLS(y, X).fit(cov_type='cluster',cov_kwds={'groups': firms},use_t=True)
                pvals = model.pvalues[0:len(goodsData.columns[goodsData.columns.str.contains(indVar)])]
                coeff =  model.params[0:len(goodsData.columns[goodsData.columns.str.contains(indVar)])]
                
                # results.loc[i,'ind'] = ind


                results.loc[i,'outcomeVar'] = outcomeVar
                results.loc[i,'weatherVar'] = weatherVar
                results.loc[i,'statVar']    = statVar
                results.loc[i,'cutoffVar']  = cutoffVar
                # results.loc[i,'firmVar']    = firmVar


                results.loc[i,'lag0']       = coeff[0]
                results.loc[i,'lag1']       = coeff[1]
                results.loc[i,'lag2']       = coeff[2]
                results.loc[i,'lag3']       = coeff[3]
                results.loc[i,'lag4']       = coeff[4]

                results.loc[i,'pval0']      = pvals[0]
                results.loc[i,'pval1']      = pvals[1]
                results.loc[i,'pval2']      = pvals[2]
                results.loc[i,'pval3']      = pvals[3]
                results.loc[i,'pval4']      = pvals[4]

                
                results.to_csv("../../data/utilitiesResults_rightInds_noCtrls.csv")
                
                print( time.time() - start)

lnRevNormd ~ precip_zipquant_0.95
0.9773828983306885
lnRevNormd ~ precip_ffquant_0.95
2.520321846008301
lnRevNormd ~ precip_zipQuarterquant_0.95
3.8356997966766357
lnRevNormd ~ precip_indQuarterquant_0.95
5.075920820236206
lnRevNormd ~ temp_zipquant_0.95
6.713193893432617
lnRevNormd ~ temp_ffquant_0.95
7.9631028175354
lnRevNormd ~ temp_zipQuarterquant_0.95
9.13026475906372
lnRevNormd ~ temp_indQuarterquant_0.95
10.570092916488647
lnCostNormd ~ precip_zipquant_0.95
11.576021909713745
lnCostNormd ~ precip_ffquant_0.95
12.521055936813354
lnCostNormd ~ precip_zipQuarterquant_0.95
13.596194982528687
lnCostNormd ~ precip_indQuarterquant_0.95
14.867467880249023
lnCostNormd ~ temp_zipquant_0.95
15.853435754776001
lnCostNormd ~ temp_ffquant_0.95
16.845340728759766
lnCostNormd ~ temp_zipQuarterquant_0.95
17.825721740722656
lnCostNormd ~ temp_indQuarterquant_0.95
19.082683801651


In [15]:
print(ind)

35


In [54]:
results.to_csv("../../data/utilitiesResults_rightInds.csv")

## Industry-Specific
Go through every famafrench industry and run the regressions above.

In [9]:
cutoffVarsYr = ['0.95', '1x5Qtrs', '1x5Yrs'] # '1x5Qtrs',
weatherVars  = ['precip_', 'temp_']        #, 'temp5Days_', 'precip5Days_'] # , 'precip_']#, , ] #[,]
statVarsYr   = ['zipquant_', 'ffquant_', 'zipQuarterquant_', 'indQuarterquant_']
outcomeVars  = ['lnRevNormd', 'lnCostNormd', 'lnRev', 'lnCost', 'revenueChange', 'costChange']

industries = range(27,44)

start = time.time()

# results = pd.DataFrame()

# i = 0
for ind in industries:
    print('##########################################################')
    print(ind)
    filename = '../../data/companyData/igData_ind' + str(ind) + '.csv'           
    goodsData = pd.read_csv(filename).drop(columns = {'Unnamed: 0'})

    
    for outcomeVar in outcomeVars:
        for weatherVar in weatherVars:
            for statVar in statVarsYr:                     
                for cutoffVar in cutoffVarsYr:
                    
                    i = i + 1
                    
                    
                    
                    goodsData = goodsData[~goodsData.lnRev.isna() & 
                                         ~goodsData.lnCost.isna()]
                    
                    
                    indVar = weatherVar + statVar + cutoffVar


                    print(outcomeVar, "~", indVar)


                    # find: concurrent ; or lagged supplier data
                    X = goodsData.loc[:,((goodsData.columns.str.contains(indVar)) | 
                                                    (goodsData.columns.str.contains('indQtr_')) |
                                                    (goodsData.columns.str.contains('gvkey_'))  | 
                                                    (goodsData.columns.str.contains('ageQtr_')) |
                                                    (goodsData.columns.str.contains('sizeQtr_')) |
                                                    (goodsData.columns.str.contains('profitQtr_')))]


                    firms = goodsData['gvkey']


                    y = goodsData[outcomeVar]


                    model = sm.OLS(y, X).fit(cov_type='cluster',cov_kwds={'groups': firms},use_t=True)
                    pvals = model.pvalues[0:len(goodsData.columns[goodsData.columns.str.contains(indVar)])]
                    coeff =  model.params[0:len(goodsData.columns[goodsData.columns.str.contains(indVar)])]

                    # results.loc[i,'ind'] = ind

                    results.loc[i,'industry'] = ind

                    results.loc[i,'outcomeVar'] = outcomeVar
                    results.loc[i,'weatherVar'] = weatherVar
                    results.loc[i,'statVar']    = statVar
                    results.loc[i,'cutoffVar']  = cutoffVar
                    # results.loc[i,'firmVar']    = firmVar


                    results.loc[i,'lag0']       = coeff[0]
                    results.loc[i,'lag1']       = coeff[1]
                    results.loc[i,'lag2']       = coeff[2]
                    results.loc[i,'lag3']       = coeff[3]
                    results.loc[i,'lag4']       = coeff[4]

                    results.loc[i,'pval0']      = pvals[0]
                    results.loc[i,'pval1']      = pvals[1]
                    results.loc[i,'pval2']      = pvals[2]
                    results.loc[i,'pval3']      = pvals[3]
                    results.loc[i,'pval4']      = pvals[4]


                    results.to_csv("../../results_byInds_withControls.csv")

                    print( time.time() - start)

##########################################################
27
lnRevNormd ~ precip_zipquant_0.95
0.2568812370300293
lnRevNormd ~ precip_zipquant_1x5Qtrs
0.48613715171813965
lnRevNormd ~ precip_zipquant_1x5Yrs
0.9219691753387451
lnRevNormd ~ precip_ffquant_0.95
1.2301170825958252
lnRevNormd ~ precip_ffquant_1x5Qtrs
1.5227131843566895
lnRevNormd ~ precip_ffquant_1x5Yrs
1.812455177307129
lnRevNormd ~ precip_zipQuarterquant_0.95
2.1013259887695312
lnRevNormd ~ precip_zipQuarterquant_1x5Qtrs
2.392078161239624
lnRevNormd ~ precip_zipQuarterquant_1x5Yrs
2.6231229305267334
lnRevNormd ~ precip_indQuarterquant_0.95
2.9388740062713623
lnRevNormd ~ precip_indQuarterquant_1x5Qtrs
3.2095448970794678
lnRevNormd ~ precip_indQuarterquant_1x5Yrs
3.4467990398406982
lnRevNormd ~ temp_zipquant_0.95
3.759603977203369
lnRevNormd ~ temp_zipquant_1x5Qtrs
4.089143991470337
lnRevNormd ~ temp_zipquant_1x5Yrs
4.368511915206909
lnRevNormd ~ temp_ffquant_0.95
4.672713041305542
lnRevNormd ~ temp_ffquant_1x5Qtrs
5.0439

43.73066687583923
lnRevNormd ~ precip_zipquant_1x5Yrs
44.795494079589844
lnRevNormd ~ precip_ffquant_0.95
45.36084604263306
lnRevNormd ~ precip_ffquant_1x5Qtrs
46.10048007965088
lnRevNormd ~ precip_ffquant_1x5Yrs
46.687483072280884
lnRevNormd ~ precip_zipQuarterquant_0.95
47.30486011505127
lnRevNormd ~ precip_zipQuarterquant_1x5Qtrs
48.05560493469238
lnRevNormd ~ precip_zipQuarterquant_1x5Yrs
48.61819314956665
lnRevNormd ~ precip_indQuarterquant_0.95
49.29151797294617
lnRevNormd ~ precip_indQuarterquant_1x5Qtrs
49.90335011482239
lnRevNormd ~ precip_indQuarterquant_1x5Yrs
50.575655937194824
lnRevNormd ~ temp_zipquant_0.95
51.23149800300598
lnRevNormd ~ temp_zipquant_1x5Qtrs
51.8380331993103
lnRevNormd ~ temp_zipquant_1x5Yrs
52.45204186439514
lnRevNormd ~ temp_ffquant_0.95
52.96769094467163
lnRevNormd ~ temp_ffquant_1x5Qtrs
53.51347804069519
lnRevNormd ~ temp_ffquant_1x5Yrs
54.07336711883545
lnRevNormd ~ temp_zipQuarterquant_0.95
54.737436056137085
lnRevNormd ~ temp_zipQuarterquant_1x5Qt

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (17,45) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


135.33137202262878
lnRevNormd ~ precip_zipquant_1x5Qtrs
136.01357007026672
lnRevNormd ~ precip_zipquant_1x5Yrs
136.82771396636963
lnRevNormd ~ precip_ffquant_0.95
137.3844599723816
lnRevNormd ~ precip_ffquant_1x5Qtrs
137.97388911247253
lnRevNormd ~ precip_ffquant_1x5Yrs
138.48277807235718
lnRevNormd ~ precip_zipQuarterquant_0.95
139.0576229095459
lnRevNormd ~ precip_zipQuarterquant_1x5Qtrs
139.6004331111908
lnRevNormd ~ precip_zipQuarterquant_1x5Yrs
140.1674120426178
lnRevNormd ~ precip_indQuarterquant_0.95
140.6954391002655
lnRevNormd ~ precip_indQuarterquant_1x5Qtrs
141.45099997520447
lnRevNormd ~ precip_indQuarterquant_1x5Yrs
142.15053582191467
lnRevNormd ~ temp_zipquant_0.95
142.9135479927063
lnRevNormd ~ temp_zipquant_1x5Qtrs
143.4477858543396
lnRevNormd ~ temp_zipquant_1x5Yrs
144.12408304214478
lnRevNormd ~ temp_ffquant_0.95
144.71761989593506
lnRevNormd ~ temp_ffquant_1x5Qtrs
145.31001591682434
lnRevNormd ~ temp_ffquant_1x5Yrs
145.9193890094757
lnRevNormd ~ temp_zipQuarterquant_

220.4457700252533
lnRevNormd ~ precip_ffquant_0.95
220.80601000785828
lnRevNormd ~ precip_ffquant_1x5Qtrs
221.20508003234863
lnRevNormd ~ precip_ffquant_1x5Yrs
221.56286907196045
lnRevNormd ~ precip_zipQuarterquant_0.95
221.9742488861084
lnRevNormd ~ precip_zipQuarterquant_1x5Qtrs
222.33070421218872
lnRevNormd ~ precip_zipQuarterquant_1x5Yrs
222.67113494873047
lnRevNormd ~ precip_indQuarterquant_0.95
223.06469917297363
lnRevNormd ~ precip_indQuarterquant_1x5Qtrs
223.41248106956482
lnRevNormd ~ precip_indQuarterquant_1x5Yrs
223.7799768447876
lnRevNormd ~ temp_zipquant_0.95
224.18055510520935
lnRevNormd ~ temp_zipquant_1x5Qtrs
224.5357871055603
lnRevNormd ~ temp_zipquant_1x5Yrs
225.00777101516724
lnRevNormd ~ temp_ffquant_0.95
225.3702609539032
lnRevNormd ~ temp_ffquant_1x5Qtrs
225.79551005363464
lnRevNormd ~ temp_ffquant_1x5Yrs
226.23985719680786
lnRevNormd ~ temp_zipQuarterquant_0.95
226.59255814552307
lnRevNormd ~ temp_zipQuarterquant_1x5Qtrs
226.93963599205017
lnRevNormd ~ temp_zipQu

...

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


4503.591308116913
lnRevNormd ~ precip_zipquant_1x5Qtrs
4505.2303240299225
lnRevNormd ~ precip_zipquant_1x5Yrs
4506.826709985733
lnRevNormd ~ precip_ffquant_0.95
4508.451539039612
lnRevNormd ~ precip_ffquant_1x5Qtrs
4510.061944961548
lnRevNormd ~ precip_ffquant_1x5Yrs
4511.672360897064
lnRevNormd ~ precip_zipQuarterquant_0.95
4513.3248970508575
lnRevNormd ~ precip_zipQuarterquant_1x5Qtrs
4514.968731164932
lnRevNormd ~ precip_zipQuarterquant_1x5Yrs
4516.644498825073
lnRevNormd ~ precip_indQuarterquant_0.95
4518.280301094055
lnRevNormd ~ precip_indQuarterquant_1x5Qtrs
4519.945167064667
lnRevNormd ~ precip_indQuarterquant_1x5Yrs
4521.552886009216
lnRevNormd ~ temp_zipquant_0.95
4523.131452083588
lnRevNormd ~ temp_zipquant_1x5Qtrs
4524.741101026535
lnRevNormd ~ temp_zipquant_1x5Yrs
4526.3714780807495
lnRevNormd ~ temp_ffquant_0.95
4527.946649074554
lnRevNormd ~ temp_ffquant_1x5Qtrs
4529.487335205078
lnRevNormd ~ temp_ffquant_1x5Yrs
4531.088220119476
lnRevNormd ~ temp_zipQuarterquant_0.95
45

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


lnRevNormd ~ precip_zipquant_0.95
4740.7354328632355
lnRevNormd ~ precip_zipquant_1x5Qtrs
4742.722780942917
lnRevNormd ~ precip_zipquant_1x5Yrs
4744.790708065033
lnRevNormd ~ precip_ffquant_0.95
4746.802354812622
lnRevNormd ~ precip_ffquant_1x5Qtrs
4748.728738069534
lnRevNormd ~ precip_ffquant_1x5Yrs
4750.809354066849
lnRevNormd ~ precip_zipQuarterquant_0.95
4752.764783143997
lnRevNormd ~ precip_zipQuarterquant_1x5Qtrs
4754.770037889481
lnRevNormd ~ precip_zipQuarterquant_1x5Yrs
4756.9438400268555
lnRevNormd ~ precip_indQuarterquant_0.95
4758.917036056519
lnRevNormd ~ precip_indQuarterquant_1x5Qtrs
4760.900903940201
lnRevNormd ~ precip_indQuarterquant_1x5Yrs
4763.040697097778
lnRevNormd ~ temp_zipquant_0.95
4765.064759016037
lnRevNormd ~ temp_zipquant_1x5Qtrs
4766.99520111084
lnRevNormd ~ temp_zipquant_1x5Yrs
4769.074250936508
lnRevNormd ~ temp_ffquant_0.95
4771.156527042389
lnRevNormd ~ temp_ffquant_1x5Qtrs
4773.118796110153
lnRevNormd ~ temp_ffquant_1x5Yrs
4775.014865159988
lnRevNorm

5033.172142982483
lnRevNormd ~ precip_ffquant_1x5Qtrs
5034.082751989365
lnRevNormd ~ precip_ffquant_1x5Yrs
5034.959795951843
lnRevNormd ~ precip_zipQuarterquant_0.95
5035.850254058838
lnRevNormd ~ precip_zipQuarterquant_1x5Qtrs
5036.78826212883
lnRevNormd ~ precip_zipQuarterquant_1x5Yrs
5037.677146196365
lnRevNormd ~ precip_indQuarterquant_0.95
5038.77793097496
lnRevNormd ~ precip_indQuarterquant_1x5Qtrs
5040.008301973343
lnRevNormd ~ precip_indQuarterquant_1x5Yrs
5041.485357999802
lnRevNormd ~ temp_zipquant_0.95
5042.432282209396
lnRevNormd ~ temp_zipquant_1x5Qtrs
5043.347914934158
lnRevNormd ~ temp_zipquant_1x5Yrs
5044.275053024292
lnRevNormd ~ temp_ffquant_0.95
5045.205983877182
lnRevNormd ~ temp_ffquant_1x5Qtrs
5046.144958972931
lnRevNormd ~ temp_ffquant_1x5Yrs
5047.033602952957
lnRevNormd ~ temp_zipQuarterquant_0.95
5047.952567100525
lnRevNormd ~ temp_zipQuarterquant_1x5Qtrs
5048.933773994446
lnRevNormd ~ temp_zipQuarterquant_1x5Yrs
5049.8648591041565
lnRevNormd ~ temp_indQuarterq

# Faster and More Heuristic

The below gives us unclustered standard errors, output to a csv file.

In [ ]:
def findSE(X,reg,y):
    N = len(X)
    p = len(X.columns) + 1  # plus one because LinearRegression adds an intercept term

    X_with_intercept = np.empty(shape=(N, p), dtype=np.float)
    X_with_intercept[:, 0] = 1
    X_with_intercept[:, 1:p] = X.values

    y_hat = reg.predict(X)
    residuals = y.values - y_hat
    residual_sum_of_squares = residuals.T @ residuals
    sigma_squared_hat = residual_sum_of_squares / (N - p)
    var_beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) * sigma_squared_hat

    se0 = var_beta_hat[1, 1] ** 0.5
    se1 = var_beta_hat[2, 2] ** 0.5
    se2 = var_beta_hat[3, 3] ** 0.5
    se3 = var_beta_hat[4, 4] ** 0.5
    se4 = var_beta_hat[5, 5] ** 0.5
    se5 = var_beta_hat[6, 6] ** 0.5
    '''se6 = var_beta_hat[7, 7] ** 0.5
    se7 = var_beta_hat[8, 8] ** 0.5
    se8 = var_beta_hat[9, 9] ** 0.5'''
    return([abs(reg.coef_[0]/se0),abs(reg.coef_[1]/se1),abs(reg.coef_[2]/se2),
            abs(reg.coef_[3]/se3),abs(reg.coef_[4]/se4),abs(reg.coef_[5]/se5)]
          )

'''        
abs(reg.coef_[0]/se0),
          abs(reg.coef_[1]/se1),
          abs(reg.coef_[2]/se2),
          abs(reg.coef_[3]/se3),
          abs(reg.coef_[4]/se4),
          abs(reg.coef_[5]/se5),
          "SE0: ", se0,
          "SE1: ", se1,
          "SE2: ", se2,
          "SE3: ", se3,
          "SE4: ", se4,
          "SE5: ", se5,

'''


In [ ]:
goodsData.columns[0:75]

In [ ]:
goodsData.columns[0:50]

In [ ]:
inds = [1, 2, 6, 7, 18, 31, 41, 42]

for ind in inds:

    tempData = goodsData[goodsData.famafrench == ind]
    print(tempData.shape)


In [ ]:
'''cutoffVarsYr = ['0.95'] # ,'1xYr']                                    #,'1x5Yrs'] #, ] # ,'1xQtr', '1x5Qtrs'
weatherVars  = ['precip_', 'temp_', 'precip5Days_', 'temp5Days_'] #[,]
statVarsYr   = ['zipquant_','zipQuarterquant_']
outcomeVars  = ['lnRev', 'revenueChange'] # ,'lnCost',  'costChange'] # [,'lnRevNormd','lnCostNormd'] # 'revenueChange' 'costChange',
firmVars     = ['firmQtr_'] # 'gvkey'
'''

# try this by industry
cutoffVarsYr = ['0.95'] # ,'1xYr']                                    #,'1x5Yrs'] #, ] # ,'1xQtr', '1x5Qtrs'
weatherVars  = ['precip_', 'temp_', 'precip5Days_', 'temp5Days_'] #[,]
statVarsYr   = ['ffquant_','indQuarterquant_']
outcomeVars  = ['lnRev', 'revenueChange',  'lnCost',  'costChange'] # [,'lnRevNormd','lnCostNormd'] # 'revenueChange' 'costChange',
firmVars     = ['firmQtr_']


inds = [1, 2, 6, 7, 18, 31, 41, 42]

goodsData = goodsData[~goodsData.lnRev.isna() & ~goodsData.lnCost.isna() &
                      ~goodsData.lnCostNormd.isna() & ~goodsData.lnRevNormd.isna()]

start = time.time()

results = pd.DataFrame()
i = 0
for ind in inds:
    print('#######################################################################################',ind)
    for outcomeVar in outcomeVars:
        for weatherVar in weatherVars:
            for statVar in statVarsYr:                     
                for cutoffVar in cutoffVarsYr:
                    for firmVar in firmVars:
                        tempData = goodsData[goodsData.famafrench == ind]
                        
                        i = i + 1
                        indVar = weatherVar + statVar + cutoffVar


                        print(outcomeVar, "~", indVar, "|", firmVar)


                        # find: concurrent ; or lagged supplier data
                        X = tempData.loc[:,((tempData.columns.str.contains(indVar)) |
                                          (tempData.columns.str.contains('indQtr_')) |
                                          # (goodsData.columns.str.contains('gvkey_'))) |   # &   
                                          # (goodsData.columns.str.contains('firmQtr_'))) |
                                          (tempData.columns.str.contains(firmVar)))] # |
                        '''(tempData.columns.str.contains('ageQtr_')) |
                          (tempData.columns.str.contains('sizeQtr_')) |
                          (tempData.columns.str.contains('profitQtr_'))]   #  & '''

                                          # (goodsData.columns.str.contains('firmQtr_')))       & 
                                        # ~(goodsData.columns.str.contains('lag4')) &
                                                                        # ~(goodsData.columns.str.contains('lag2')) & 


                        X = X[X.columns[(X.sum(axis = 0) >= 4)]]
                        # print(X.columns)
                        firms = tempData['gvkey']


                        y = tempData[outcomeVar]


                        ######################################
                        # fit the model on this subset
                        reg = linear_model.LinearRegression()
                        reg.fit(X,y)


                        # print('Coeff: ' , reg.coef_[0:5], 'SE type (looking >2): ', findSE(X,reg,y))
                        results.loc[i,'ind'] = ind


                        results.loc[i,'outcomeVar'] = outcomeVar
                        results.loc[i,'weatherVar'] = weatherVar
                        results.loc[i,'statVar']    = statVar
                        results.loc[i,'cutoffVar']  = cutoffVar
                        results.loc[i,'firmVar']    = firmVar


                        results.loc[i,'lag0']       = reg.coef_[0]
                        results.loc[i,'lag1']       = reg.coef_[1]
                        results.loc[i,'lag2']       = reg.coef_[2]
                        results.loc[i,'lag3']       = reg.coef_[3]
                        results.loc[i,'lag4']       = reg.coef_[4]



                        seratios = findSE(X,reg,y)

                        results.loc[i,'ratio0']       = seratios[0]
                        results.loc[i,'ratio1']       = seratios[1]
                        results.loc[i,'ratio2']       = seratios[2]
                        results.loc[i,'ratio3']       = seratios[3]
                        results.loc[i,'ratio4']       = seratios[4]

                        # print(results)

                        print(time.time() - start)

                        print('*******************************************************************')
                    
results.to_csv("../../data/results_notNormd.csv")

In [ ]:
i = 0

results = pd.DataFrame()
results.loc[i,'outcome'] = 0
results

# Indirect Effects
This is almost exactly the same but with supplier information in place of the direct company information.

In [ ]:
cutoffVarsYr = ['1xYr'] #, ] # ,'1xQtr''1x5Qtrs',
weatherVars  = ['precip_', 'temp_', 'precip5Days_', 'temp5Days_'] #[,]
statVarsYr   = ['annualquant_']  #,'zipQuarterquant_']
outcomeVars  = ['costChange', 'revenueChange', 'lnRevNormd', 'lnCostNormd'] # ['revenueChange'] #[, 'costChange']#,'lnCost','lnInc','lnRev']

goodsData = goodsData[~goodsData.lnRev.isna() & ~goodsData.lnCost.isna() & ~goodsData.lnCostNormd.isna()]
goodsData['scTercile']  = pd.qcut(goodsData['suppliers'], 3, labels=False, duplicates = 'drop')


start = time.time()

for outcomeVar in outcomeVars:
    for weatherVar in weatherVars:
        for statVar in statVarsYr:                     
            for cutoffVar in cutoffVarsYr:
                indVar = weatherVar + statVar + cutoffVar
                
                
                print(outcomeVar, "~", indVar)


                # find: concurrent ; or lagged supplier data
                X = goodsData.loc[:,(((goodsData.columns.str.contains(indVar)) & goodsData.columns.str.contains('supplier')) | 
                                                (goodsData.columns.str.contains('indQtr_')) |
                                                (goodsData.columns.str.contains('gvkey_'))  | 
                                                (goodsData.columns.str.contains('ageQtr_')) |
                                                (goodsData.columns.str.contains('sizeQtr_')) |
                                                (goodsData.columns.str.contains('profitQtr_')) | 
                                                (goodsData.columns == 'suppliers')) &   
                                                # (goodsData.columns.str.contains('firmQtr_')))       & 
                                                ~(goodsData.columns.str.contains('gvkey_1013')) & # do 1 minus the FEs
                                                ~(goodsData.columns.str.contains('indQtr_12010_1'))]# & 
                                                # ~(goodsData.columns.str.contains('lag4'))]     
                
                
                firms = goodsData['gvkey']


                y = goodsData[outcomeVar]
                
                
                results = sm.OLS(y, X).fit(cov_type='cluster',cov_kwds={'groups': firms},use_t=True)
                pvals = results.pvalues[0:len(goodsData.columns[goodsData.columns.str.contains(indVar) & goodsData.columns.str.contains('supplier_')])]
                coeff = results.params[0:len(goodsData.columns[goodsData.columns.str.contains(indVar)  & goodsData.columns.str.contains('supplier_')])]
                print(coeff)
                print(pvals)
                
                
                print( time.time() - start)
                
                
                

In [ ]:
cutoffVarsYr = ['1xYr'] #, ] # ,'1xQtr''1x5Qtrs',
weatherVars  = ['precip_', 'temp_', 'precip5Days_', 'temp5Days_'] #[,]
statVarsYr   = ['annualquant_']  #,'zipQuarterquant_']
outcomeVars  = ['revenueChange',  'costChange'] # ['revenueChange'] #['incomeChange', 'revenueChange', 'costChange']#,'lnCost','lnInc','lnRev']


# 'revenueChange',  'costChange', 
# 'lnRevNormd', 'lnCostNormd'


goodsData = goodsData[~goodsData.lnRev.isna() & ~goodsData.lnCost.isna() & ~goodsData.lnCostNormd.isna()]


start = time.time()

for outcomeVar in outcomeVars:
    for weatherVar in weatherVars:
        for statVar in statVarsYr:                     
            for cutoffVar in cutoffVarsYr:
                indVar = weatherVar + statVar + cutoffVar
                supplierIndVar = 'supplier_' + weatherVar + statVar + cutoffVar
                
                
                print(outcomeVar, "~", indVar, supplierIndVar)


                # find: concurrent ; or lagged supplier data
                X = goodsData.loc[:,((goodsData.columns.str.contains(indVar) & goodsData.columns.str.contains('supplier')) |
                                  (goodsData.columns.str.contains('indQtr_'))       |
                                  (goodsData.columns.str.contains('gvkey_'))) &   
                                # (goodsData.columns.str.contains('firmQtr_')))       & 
                                  ~(goodsData.columns.str.contains('gvkey_1045'))     & # do 1 minus the FEs
                                ~(goodsData.columns.str.contains('indQtr_12013_1'))]#  &
                                # ~(goodsData.columns.str.contains('lag4')) &
                                # (goodsData.columns.str.contains('ageQtr_')) |
                                # (goodsData.columns.str.contains('sizeQtr_')) |
                                # (goodsData.columns.str.contains('profitQtr_'))) & 
                                # ~(goodsData.columns.str.contains('firmQtr_10451'))] # & # do 1 minus the FEs
                                # ~(goodsData.columns.str.contains('indQtr_12013_1'))]# & 
                                # ~(goodsData.columns.str.contains('lag2')) & 
                                 
                                
                print(X.columns[0:5])

                X = X[X.columns[(X.sum(axis = 0) >= 4)]]
                firms = goodsData['gvkey']


                y = goodsData[outcomeVar]


                ######################################
                # fit the model on this subset
                reg = linear_model.LinearRegression()
                reg.fit(X,y)


                print('Coeff: ' , reg.coef_[0:5], 'SE type (looking >2): ', findSE(X,reg,y))
                
                print(time.time() - start)
                
                print('*******************************************************************')
                


# Quarterly-Specific Effects
Look at what is going on within a quarter, for a specific zipcode. 

Note: this is effectively deprecated at this point because the zip-level effects mask the differences in the underlying weather distributions, ie some regions would likely benefit from getting warmer. If we want to do a zip-level study, we should also include some sort of threshold here.

Do this for all qtrs.

In [ ]:
cutoffVarsQtr = ['0.95','1xQtr','1x5Qtrs']# ,'1x5Qtrs'] # ,
weatherVars   = ['precip5Days_'] # ['precip5Days_','temp5Days_']
statVarQtr    = 'zipQuarterquant_'
outcomeVars   = ['revenueChange'] #['incomeChange', 'revenueChange', 'costChange']



for weatherVar in weatherVars:
    for cutoffVar in cutoffVarsQtr:
        indVar = weatherVar + statVarQtr + cutoffVar            
        for outcomeVar in outcomeVars:
            print(outcomeVar, "~", indVar)



            X = goodsData.loc[:,(goodsData.columns == indVar) | 
                              (goodsData.columns.str.contains('indQtr_')) |
                              (goodsData.columns.str.contains('gvkey_')) & 
                             ~(goodsData.columns.str.contains('gvkey_1004')) & # do 1 minus the FEs
                             ~(goodsData.columns.str.contains('indQtr_422019_4'))]


            X = X[X.columns[(X.sum(axis = 0) >= 4)]]
            firms = goodsData['gvkey']


            y = goodsData[outcomeVar]


            ######################################
            # fit the model on this subset
            reg = linear_model.LinearRegression()
            reg.fit(X,y)


            print('Coeff: ' , reg.coef_[0:5], 'SE type (looking >2): ', findSE(X,reg,y))
            print(time.time() - start)

In [ ]:
X[X.columns[(X.sum(axis = 0) >= 4)]].shape